# ABS Monthly International Trade 5368

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_plot_constants,
    get_meta_constants,
    get_abs_meta_and_data,
    get_identifier,
    clear_cache,
)
from plotting import (
    finalise_plot,
    recalibrate,
    clear_chart_dir,
    set_chart_dir,
    seas_trend_plot,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
# in case of emergency
if False:
    clear_cache()

In [3]:
catalogue_id = "5368"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_trade = get_abs_meta_and_data(catalogue_id, verbose=False)
if abs_trade is None:
    sys.exit(-1)
meta_trade = abs_trade[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta_trade)
did_col, id_col, table_col, type_col, unit_col, tdesc_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Source modification date: 2023-12-06 04:59:38+00:00
Retrieving data from the cache file: ABS_CACHE/7baac751e63e8f015065399fd66b22d0--Time-series-spreadsheets-all.zip
Extracting DataFrames from the zip-file ...
Caution: Could not find the "Index" sheet in 5368092.xlsx


## Plot

### Initialisation


In [4]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Headline charts

In [5]:
table = "1"
headline = meta_trade[(meta_trade[table_col] == table)][did_col].unique()

for h in headline:
    # secure the data
    f = pd.DataFrame()
    for service_type in "Seasonally Adjusted", "Trend":
        id, units = get_identifier(meta_trade, h, service_type, table)
        f[service_type] = abs_trade[table][id]
    f, units = recalibrate(f, units, verbose=True)

    # plot
    title = (
        h.replace("Credits", "Credits (Exports)")
        .replace("Debits", "Debits (Imports)")
        .replace(" ;", "")
        .strip()
    )
    LONG_LINE = 50
    if "," in title and len(title) > LONG_LINE:
        title = title.replace(",", ",\n")
    lfooter = "Australia. Current Prices. "
    if f["Trend"].isna().iloc[-1]:
        last = f["Trend"].last_valid_index()
        lfooter += f"No trend data after {last}. "

    seas_trend_plot(
        f,
        starts=plot_times,
        tags=plot_tags,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=lfooter,
        show=SHOW,
    )

recalibrate(): No adjustments needed


## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Dec 21 2023 20:46:15

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.18.1

matplotlib: 3.8.2
sys       : 3.11.7 | packaged by conda-forge | (main, Dec 15 2023, 08:41:36) [Clang 16.0.6 ]
pandas    : 2.1.4

Watermark: 2.4.3



In [7]:
print("Finished")

Finished
